In [10]:
import my_ast
my_ast.process_prompt('./prompt-base.md', """
You play the role of a software architect, AI expert, and telecommunications expert to answer questions for me.
使用langchain 和 bingsearch 实现一个问答系统
""")


/home/weiz/.py/venvs/llm/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in LLMConfig has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


```thinking
Hmm, this is an interesting question about building a QA system with LangChain and Bing Search. Let me think through the key aspects...

First, let's consider what we're trying to achieve here - essentially creating a system that can search the web via Bing and provide relevant answers using LLM capabilities. This combines web search, information extraction, and natural language generation.

The core components we'll need:
- LangChain as the orchestration framework
- Bing Search API for retrieving relevant web content 
- An LLM (probably GPT or similar) to process and generate answers
- Some way to manage the conversation flow and context

Key considerations coming to mind:
For Bing Search integration, we'll need proper API credentials and rate limiting handling. The search results will need cleaning and preprocessing before feeding to the LLM.

For the LangChain implementation, we should leverage its tools and agents capabilities. The RetrievalQA chain would be perfect for

https://python.langchain.com/docs/integrations/tools/bing_search/

In [4]:
from langchain_community.utilities import BingSearchAPIWrapper

search = BingSearchAPIWrapper(k=4)

search.run("python")



'Experienced programmers in any other language can pick up <b>Python</b> very quickly, and beginners find the clean syntax and indentation structure easy to learn. Whet your appetite with our <b>Python</b> 3 overview. Installer packages for <b>Python</b> on macOS downloadable from <b>python</b>.org are signed with with an Apple Developer ID Installer certificate. As of <b>Python</b> 3.11.4 and 3.12.0b1 (2023-05-23), release installer packages are signed with certificates issued to the <b>Python</b> Software Foundation (Apple Developer ID BMM5U3QVKW) ). <b>Python</b> <b>基础教程</b> Python 是一种<b>解释型</b>、面向对象、动态数据类型的高级程序设计语言。 Python 由 Guido van Rossum 于 1989 年底发明，第一个公开发行版发行于 1991 年。 像 Perl 语言一样, Python 源代码同样遵循 GPL(GNU General Public License) 协议。 <b>Python由荷兰</b>国家<b>数学</b>与计算机科学研究中心的吉多·范罗苏姆于1990年代初设计，作为一门叫做ABC语言的替代品。Python提供了高效的高级数据结构，还能简单有效地面向对象编程。'

In [1]:
from langchain.utilities import BingSearchAPIWrapper
from langchain_community.tools.bing_search import BingSearchResults
from dotenv import load_dotenv, find_dotenv
import json
import os

load_dotenv(find_dotenv(),override=True)
# 初始化 Bing 搜索
api_wrapper = BingSearchAPIWrapper(
    bing_search_url=os.getenv("BING_SEARCH_URL", ""),
    bing_subscription_key=os.getenv("BING_SEARCH_API_KEY", ""),
    k=4
)

# 创建搜索工具
tool = BingSearchResults(api_wrapper=api_wrapper)
question = "什么是量子计算"

# .invoke wraps utility.results
response = tool.invoke(question)
response = json.loads(response.replace("'", '"'))
for item in response:
    print(item)


{'snippet': '量子计算是一<b>种遵循量子力学规律调控量子信息单元进行计算的新型计算模式</b>。<b>与</b>经典计算不同，量子计算遵循量子力学规律，它是能突破经典算力瓶颈的新型计算模式。量子计算机，作为执行量子计算任务的设备，以量子比特（qubit）为基本运算单元。', 'title': '<b>量子计算</b>（遵循<b>量子</b>力学规律的<b>计算</b>模式）_百度百科', 'link': 'https://baike.baidu.com/item/%E9%87%8F%E5%AD%90%E8%AE%A1%E7%AE%97/11035661'}
{'snippet': '量子计算是利用亚原子粒子的物理学原理来执行并行计算的系统，可以在某些数学运算中更快速有效地模拟离奇的量子现象。本文介绍了量子计算的概念、作用、工作原理和现状，以及 NVIDIA cuQuantum SDK 的应用。', 'title': '科普 | <b>什么是量子计算</b>？ - 知乎', 'link': 'https://zhuanlan.zhihu.com/p/366657266'}
{'snippet': '本文介绍了量子计算的基本概念、类型、优势和应用领域，以及全球量子计算产业的发展现状和前景。量子计算是利用量子力学的基本原理来加速解决复杂计算的过程，可以在各个行业提高效率和创新能力，但也面临着技术和市场的挑战。', 'title': '一文看懂<b>量子计算</b>：原理、应用、行业现状-36氪', 'link': 'https://www.36kr.com/p/1723178598401'}
{'snippet': '<b>量子计算利用专门技术</b>（包括计算机硬件和利用量子力学的算法）来解决传统计算机或超级计算机无法解决或无法快速解决的复杂问题。 如今，IBM Quantum 为数十万开发人员提供了真正的量子硬件，这是一种科学家在三十年前才开始想象的工具。', 'title': '<b>什么是量子计算</b>？| IBM', 'link': 'https://www.ibm.com/cn-zh/topics/quantum-computing'}


In [5]:
import os
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(),override=True)

api_key=os.getenv("ONEAPI_API_KEY", "")
base_url=os.getenv("ONEAPI_BASE_URL", "")

instructions = """You are an assistant."""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)

llm = ChatOpenAI(
    api_key=api_key, 
    base_url=base_url, 
    model="gpt-4o"
    # model="claude-3-5-sonnet"
)
api_wrapper = BingSearchAPIWrapper(
    bing_search_url=os.getenv("BING_SEARCH_URL", ""),
    bing_subscription_key=os.getenv("BING_SEARCH_API_KEY", ""),
    k=4
)

# 创建搜索工具
tool = BingSearchResults(api_wrapper=api_wrapper)
tools = [tool]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)
agent_executor.invoke({"input": "什么是量子计算?，量子计算和量子力学有什么关系？"})



/home/weiz/.py/venvs/llm/lib/python3.11/site-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
量子计算是一种基于量子力学原理的新型计算方式。传统计算机使用比特（0和1）作为基本单位，而量子计算机则使用量子比特（qubits）。量子比特可以同时处于0和1的叠加状态，这使得量子计算机在处理某些复杂问题时具有显著的优势。

量子计算和量子力学的关系主要体现在以下几个方面：

1. **叠加原理**：这是量子力学的一个基本原则，允许量子比特同时处于多种状态。这使得量子计算机能够同时处理大量可能的计算路径。

2. **纠缠现象**：量子力学中的纠缠现象允许多个量子比特之间形成强相关性，改变一个量子比特的状态会立即影响另一个纠缠态的量子比特，即使它们相距很远。纠缠被用来实现量子计算中的高效信息传输和处理。

3. **量子门操作**：量子计算使用量子门来操控量子比特状态的变化，这些操作基于量子力学的基本原理。

4. **量子退相干**：量子计算中需要克服的问题之一是退相干，即量子信息与环境相互作用后丧失其量子特性。理解和解决退相干问题依赖于量子力学的深入研究。

总的来说，量子计算是量子力学在计算领域的应用，利用量子力学的特性来实现传统计算机无法企及的计算能力。

> Finished chain.


{'input': '什么是量子计算?，量子计算和量子力学有什么关系？',
 'output': '量子计算是一种基于量子力学原理的新型计算方式。传统计算机使用比特（0和1）作为基本单位，而量子计算机则使用量子比特（qubits）。量子比特可以同时处于0和1的叠加状态，这使得量子计算机在处理某些复杂问题时具有显著的优势。\n\n量子计算和量子力学的关系主要体现在以下几个方面：\n\n1. **叠加原理**：这是量子力学的一个基本原则，允许量子比特同时处于多种状态。这使得量子计算机能够同时处理大量可能的计算路径。\n\n2. **纠缠现象**：量子力学中的纠缠现象允许多个量子比特之间形成强相关性，改变一个量子比特的状态会立即影响另一个纠缠态的量子比特，即使它们相距很远。纠缠被用来实现量子计算中的高效信息传输和处理。\n\n3. **量子门操作**：量子计算使用量子门来操控量子比特状态的变化，这些操作基于量子力学的基本原理。\n\n4. **量子退相干**：量子计算中需要克服的问题之一是退相干，即量子信息与环境相互作用后丧失其量子特性。理解和解决退相干问题依赖于量子力学的深入研究。\n\n总的来说，量子计算是量子力学在计算领域的应用，利用量子力学的特性来实现传统计算机无法企及的计算能力。'}

In [9]:
import os
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(),override=True)

api_key=os.getenv("ONEAPI_API_KEY", "")
base_url=os.getenv("ONEAPI_BASE_URL", "")

instructions = """You are an assistant."""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)
print(prompt)

/home/weiz/.py/venvs/llm/lib/python3.11/site-packages/langsmith/client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


input_variables=['agent_scratchpad', 'input'] optional_variables=['chat_history'] input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')]

https://python.langchain.com/docs/integrations/llms/anthropic/

In [ ]:
%pip install -qU langchain-anthropic

In [ ]:
from langchain_anthropic import AnthropicLLM
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv, find_dotenv
import os
load_dotenv(find_dotenv(),override=True)

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate.from_template(template)

api_key=os.getenv("ONEAPI_API_KEY", "")
base_url=os.getenv("ONEAPI_BASE_URL", "")

model = AnthropicLLM(
    api_key=api_key, 
    base_url=base_url, 
    model="claude-3-5-sonnet"
)

chain = prompt | model

chain.invoke({"question": "What is LangChain?"})

In [8]:
import os
from dotenv import load_dotenv, find_dotenv
from langchain.utilities import BingSearchAPIWrapper
from langchain.agents import Tool
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

load_dotenv(find_dotenv(),override=True)
api_key=os.getenv("ONEAPI_API_KEY", "")
base_url=os.getenv("ONEAPI_BASE_URL", "")

# 初始化 Bing 搜索
search = BingSearchAPIWrapper(
    bing_search_url=os.getenv("BING_SEARCH_URL", ""),
    bing_subscription_key=os.getenv("BING_SEARCH_API_KEY", ""),
    # k=4
)

# 创建搜索工具
search_tool = Tool(
    name="Bing Search",
    description="用于搜索互联网信息的工具",
    func=search.run
)

def search(question):
    search_result = search_tool.run(question)
    return search_result

# 设置提示模板
prompt_template = """根据以下信息回答问题：

搜索结果: {search_result}
问题: {question}

请提供详细的答案："""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["search_result", "question"]
)

# 创建 LLM Chain

llm = ChatOpenAI(
    api_key=api_key, 
    base_url=base_url, 
    model="gpt-4o"
    # model="claude-3-5-sonnet"
)
chain = LLMChain(llm=llm, prompt=prompt)

def get_answer(question):
    try:
        # 执行搜索
        search_result = search_tool.run(question)
        
        # 生成答案
        response = chain.invoke({
            "search_result": search_result,
            "question": question
        })
        
        return response
    except Exception as e:
        return f"发生错误: {str(e)}"

# 使用示例
question = "什么是量子计算？"
answer = get_answer(question)
print(answer)

{'search_result': '量子计算是<b>一种遵循量子力学规律调控量子信息单元进行计算的新型计算模式</b>。与经典计算不同，量子计算遵循量子力学规律，它是能突破经典算力瓶颈的新型计算模式。量子计算机，作为执行量子计算任务的设备，以量子比特（qubit）为基本运算单元。 <b>量子计算使用亚原子粒子的物理学领域来执行复杂的并行计算</b>，<b>从而取代了当今计算机系统中更简单的晶体管</b>。 量子计算机使用量子比特计算，计算单元可以打开，关闭或之间的任何值，而不是传统计算机中的字符，要么打开，要么关闭，要么是 1，要么是 0。 量子比特居于中间态的能力（称为“态叠加”），这为计算方程增加了强大的功能，使量子计算机在某种数学运算中更胜一筹。 量子计算机的作用. 量子计算机可以通过量子比特进行计算，这种计算过程需要耗费传统计算机无限长的时间，有时甚至根本无法完成。 例如，如今的计算机使用 8 位表示介于 0 到 255 之间的任何数字。 得益于态叠加原理，量子计算机可以使用八个量子比特同时表示 0 到 255 之间的每个数字。 什么是量子计算？ 量子计算机<b>可以比传统计算机更有效地处理大量复杂的数据集</b>。 他们利用量子力学的基本原理来加速解决复杂计算的过程。 <b>量子计算</b>利用专门技术（包括计算机硬件和利用量子力学的算法）来解决传统计算机或超级计算机无法解决或无法快速解决的复杂问题。 如今，IBM Quantum 为数十万开发人员提供了真正的量子硬件，这是一种科学家在三十年前才开始想象的工具。 <b>量子计算是</b>一个<b>多学科领域</b>，涵盖<b>计算机科学</b>、<b>物理学和数学的各个</b>方面，它利用量子力学，可以比传统计算机更快地解决复杂问题。 量子计算领域包括硬件研究和应用程序开发。 本文介绍量子计算的原则、与经典计算的比较方式，以及它如何使用量子力学原理。 由于在经典计算机上模拟量子系统比较困难，由此产生了量子计算机的想法。 在 20 世纪 80 年代，Richard Feynman 和 Yuri Manin 分别提出，基于量子现象的硬件在模拟量子系统方面可能比传统计算机更有效。 可以通过多种方法来了解量子力学难以模拟的原因。 最简单的就是看物质在量子层面上有多种可能的构型（称为状态）。 假设电子系统有 40 

In [ ]:
from dotenv import load_dotenv, find_dotenv
import os
load_dotenv(find_dotenv(),override=True)
api_key=os.getenv("ONEAPI_API_KEY", "")
base_url=os.getenv("ONEAPI_BASE_URL", "")
print(api_key)
print(base_url)

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(api_key=api_key, base_url=base_url, model="claude-3-5-sonnet")
response = llm.stream("你是哪个版本的模型")
for chunk in response:
    print(chunk.content, end="")

In [4]:
import datetime
prompt = open('prompt-base.md', 'r', encoding='utf-8').read()
prompt = prompt + """
把下面的代码进行函数和类抽取重构，使之更简洁，易扩展，易维护，代码不需要注释
新代码应该支持修改模型名称，是否使用流式
```python
from dotenv import load_dotenv, find_dotenv
import os
import datetime

load_dotenv(find_dotenv(),override=True)
api_key=os.getenv("ONEAPI_API_KEY", "")
base_url=os.getenv("ONEAPI_BASE_URL", "")
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(api_key=api_key, base_url=base_url, model="claude-3-5-sonnet")

prompt = open('prompt-base.md', 'r', encoding='utf-8').read()
prompt = prompt + "1.8 和 1.11 谁大"
response = llm.stream(prompt)
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
file_path = f'sessions/temp-{timestamp}.md'

with open(file_path, 'w', encoding='utf-8') as file:
    for chunk in response:
        print(chunk.content, end="")
        file.write(chunk.content)
```
"""
response = llm.stream(prompt)
# 获取当前时间戳
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
# 定义文件路径
file_path = f'sessions/temp-{timestamp}.md'

# 将内容写入文件
with open(file_path, 'w', encoding='utf-8') as file:
    for chunk in response:
        # print(chunk.content, end="")
        file.write(chunk.content)